In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
sns.set_style('whitegrid')
pd.pandas.set_option('display.max_columns', None)

In [2]:
train = pd.read_csv('application_train.csv')
test = pd.read_csv('../input/home-credit-default-risk/application_test.csv')
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')
bureau_bal = pd.read_csv('../input/home-credit-default-risk/bureau_balance.csv')
pos_cash_bal = pd.read_csv('../input/home-credit-default-risk/POS_CASH_balance.csv')
credit_card_bal = pd.read_csv('../input/home-credit-default-risk/credit_card_balance.csv')
previous_app = pd.read_csv('../input/home-credit-default-risk/previous_application.csv')
installment_payments = pd.read_csv('../input/home-credit-default-risk/installments_payments.csv')

FileNotFoundError: [Errno 2] File ../input/home-credit-default-risk/application_train.csv does not exist: '../input/home-credit-default-risk/application_train.csv'

In [ ]:
X = train.drop('TARGET', axis = 1)
Y = train['TARGET']

In [ ]:
X['YEARS_BEGINEXPLUATATION_AVG'].isnull()

In [ ]:
X['AMT_REQ_CREDIT_BUREAU_YEAR'].unique()

In [ ]:
Y

In [ ]:
categorical_features = [feature for feature in X.columns if X[feature].dtypes=='O']

In [ ]:
categorical_features

In [ ]:
for feature in categorical_features:
    data=train.copy()
    data.groupby(feature)['TARGET'].mean().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('TARGET')
    plt.title(feature)
    plt.show()

In [ ]:
X.drop('WEEKDAY_APPR_PROCESS_START', axis = 1, inplace = True)

In [ ]:
X.groupby('NAME_FAMILY_STATUS').count()

In [ ]:
X[X.NAME_FAMILY_STATUS == 'Unknown']

In [ ]:
X['NAME_FAMILY_STATUS']=np.where(X['NAME_FAMILY_STATUS'] != 'Unknown',X['NAME_FAMILY_STATUS'],'Single / not married')

In [ ]:
X.groupby('CODE_GENDER').count()

In [ ]:
X['CODE_GENDER']=np.where(X['CODE_GENDER'] == 'XNA','F', X['CODE_GENDER'])

In [ ]:
X.groupby('NAME_TYPE_SUITE').count()

In [ ]:
X.NAME_TYPE_SUITE.fillna('Unaccompanied', inplace = True)

In [ ]:
temp=X.groupby('CNT_CHILDREN')['CODE_GENDER'].count()/len(X)
temp_df=temp[temp>0.05].index

In [ ]:
X['CNT_CHILDREN']=np.where(X['CNT_CHILDREN'].isin(temp_df),X['CNT_CHILDREN'],'Lots')

In [ ]:
categorical_features = [feature for feature in X.columns if X[feature].dtypes=='O']
categorical_features

In [ ]:
X.groupby('NAME_INCOME_TYPE').count()

In [ ]:
X['NAME_INCOME_TYPE']=np.where(X['NAME_INCOME_TYPE'] != 'Maternity leave',X['NAME_INCOME_TYPE'],'Unemployed')
X['NAME_INCOME_TYPE']=np.where(X['NAME_INCOME_TYPE'] == 'Businessman','Working', X['NAME_INCOME_TYPE'])
X['NAME_INCOME_TYPE']=np.where(X['NAME_INCOME_TYPE'] == 'Student','Working', X['NAME_INCOME_TYPE'])

In [ ]:
X[['CNT_FAM_MEMBERS', 'CNT_CHILDREN']].corr()

In [ ]:
X.drop('CNT_FAM_MEMBERS', axis =1, inplace = True)

In [ ]:
train[['REGION_RATING_CLIENT', 'REGION_RATING_CLIENT_W_CITY', 'TARGET']].corr()

In [ ]:
X.drop('REGION_RATING_CLIENT', axis =1, inplace = True)

In [ ]:
X.shape

In [ ]:
categorical_features = [feature for feature in X.columns if X[feature].dtypes=='O']
len(categorical_features)

In [ ]:
numerical_features = [feature for feature in X.columns if X[feature].dtypes != 'O']

In [ ]:
len(numerical_features)

In [ ]:
X[X.DAYS_EMPLOYED >0]['NAME_INCOME_TYPE'].unique()

In [ ]:
sns.distplot(X[X.DAYS_EMPLOYED <0]['DAYS_EMPLOYED'])

In [ ]:
X.groupby("NAME_INCOME_TYPE").count()

In [ ]:
X['DAYS_EMPLOYED']=np.where(X['DAYS_EMPLOYED']< 0,X['DAYS_EMPLOYED'],0)

In [ ]:
X

In [ ]:
X.isna().sum().sort_values(ascending=False).head(50)

In [ ]:
sns.distplot(np.log(train['AMT_INCOME_TOTAL']))

In [ ]:
train['REG_REGION_NOT_WORK_REGION'].sum()

In [ ]:
train['ORGANIZATION_TYPE'].nunique()

In [ ]:
sns.distplot(train['EXT_SOURCE_1'].dropna())

In [ ]:
train[['EXT_SOURCE_1', 'EXT_SOURCE_2', 'EXT_SOURCE_3']].dropna().corr()

In [ ]:
bureau['SK_ID_CURR'].nunique()

In [ ]:
bureau['CREDIT_CURRENCY'].unique()

In [ ]:
bureau['CREDIT_TYPE'].unique()

In [ ]:
bureau.groupby('CNT_CREDIT_PROLONG').count()

In [ ]:
bureau['AMT_ANNUITY'].isna().sum()

In [ ]:
bureau_bal['STATUS'].unique()

In [ ]:
installment_payments.isna().sum()

In [ ]:
installment_payments['PAYMENT_DELAY'] =  installment_payments['DAYS_ENTRY_PAYMENT'] - installment_payments['DAYS_INSTALMENT']

In [ ]:
installment_payments[installment_payments['PAYMENT_DELAY'] >365]

In [ ]:
installment_payments['PAYMENT_RATIO'] = installment_payments['AMT_PAYMENT']/(installment_payments['AMT_INSTALMENT'] + 1)

In [ ]:
installment_payments['PAYMENT_RATIO'].dropna().describe()

In [ ]:
installment_payments[installment_payments['PAYMENT_RATIO'] < 0.2]

In [ ]:
installment_payments['NUM_INSTALMENT_NUMBER'].nunique()

In [ ]:
installment_payments[installment_payments['SK_ID_CURR'] == 159586]['SK_ID_PREV'].unique()

In [ ]:
installment_payments[['NUM_INSTALMENT_NUMBER', 'PAYMENT_DELAY']]

In [ ]:
import sys
def return_size(df):
    """Return size of dataframe in gigabytes"""
    return round(sys.getsizeof(df) / 1e9, 2)

def convert_types(df, print_info = False):
    
    original_memory = df.memory_usage().sum()
    
    # Iterate through each column
    for c in df:
        
        # Convert ids and booleans to integers
        if ('SK_ID' in c):
            df[c] = df[c].fillna(0).astype(np.int32)
            
        # Convert objects to category
        elif (df[c].dtype == 'object') and (df[c].nunique() < df.shape[0]):
            df[c] = df[c].astype('category')
        
        # Booleans mapped to integers
        elif list(df[c].unique()) == [1, 0]:
            df[c] = df[c].astype(bool)
        
        # Float64 to float32
        elif df[c].dtype == float:
            df[c] = df[c].astype(np.float32)
            
        # Int64 to int32
        elif df[c].dtype == int:
            df[c] = df[c].astype(np.int32)
        
    new_memory = df.memory_usage().sum()
    
    if print_info:
        print(f'Original Memory Usage: {round(original_memory / 1e9, 2)} gb.')
        print(f'New Memory Usage: {round(new_memory / 1e9, 2)} gb.')
        
    return df

In [ ]:
train = convert_types(train)
test = convert_types(test)
bureau = convert_types(bureau)
bureau_bal = convert_types(bureau_bal)
pos_cash_bal = convert_types(pos_cash_bal)
credit_card_bal = convert_types(credit_card_bal)
previous_app = convert_types(previous_app)
installment_payments = convert_types(installment_payments)

In [ ]:
installment_payments.groupby('SK_ID_PREV').count()

In [ ]:
bureau[['SK_ID_CURR', 'DAYS_CREDIT']].groupby('SK_ID_CURR').min()

In [ ]:
bureau.groupby('SK_ID_CURR').count()

In [ ]:
#!pip install pycaret

In [ ]:
#from pycaret.classification import *


In [ ]:
#clf = setup(train, target = 'TARGET')

In [ ]:
#classification = compare_models()

In [ ]:
from sklearn.model_selection import StratifiedKFold
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score

lgb = LGBMClassifier()

skf=StratifiedKFold(n_splits=5)
skf.get_n_splits(X, Y)
accuracy=[]

for train_index, test_index in skf.split(X,Y): 
    print("Train:", train_index, "Validation:", test_index) 
    X1_train, X1_test = X.iloc[train_index], X.iloc[test_index] 
    y1_train, y1_test = Y.iloc[train_index], Y.iloc[test_index]
    
    lgb.fit(X1_train,y1_train)
    prediction=lgb.predict(X1_test)
    score= roc_auc_score(prediction,y1_test)
    accuracy.append(score)
    
print(accuracy)

np.array(accuracy).mean()

In [ ]:
bureau = pd.read_csv('../input/home-credit-default-risk/bureau.csv')

In [ ]:
a = bureau.groupby('SK_ID_CURR').count()['CREDIT_ACTIVE']

In [ ]:
b = pd.merge(X, a, on='SK_ID_CURR', how='left')

In [ ]:
b

In [ ]:
b['CREDIT_ACTIVE'].isna().sum()

In [ ]:
bureau_bal

In [ ]:
bureau['SK_ID_BUREAU'].nunique()

In [ ]:
bureau_bal.groupby('SK_ID_BUREAU').count()

In [ ]:
bureau_bal.groupby('STATUS').count()

In [ ]:
pos_cash_bal